In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import math
import urllib
import os
import os.path as osp
import nbformat
import json
import ssl
import sys
import time
import hashlib
import pickle
import io
import random
import getpass
import inspect
import socket
import tempfile
from datetime import datetime, date
from datetime import timedelta

import matplotlib
import pylab # for side effects on matplotlib
import pandas as pd
import numpy as np
import IPython.display as ipd
from datetime import datetime

In [2]:
#functions
def array_to_html(arr, row_names=None, col_names=None, title=None):
    ''' array_to_html, if placed at the end of a Anaconda cell, will
        display the array in html.   It uses pandas.
    '''
    if len(arr.shape) not in [1, 2]:
        print('The array must be a one- or two-dimensional array. This array is',
             len(arr.shape), 'dimensions')
        return

    if title:
        ipd.display(ipd.HTML('<h1>' + title + '</h1>'))
    df = pd.DataFrame(arr, index=row_names, columns=col_names)
    return ipd.display(ipd.HTML(df.to_html()))
def separate(string):
    separated=[]
    semic=[-1]
    reverse=string[::-1]
    lastword=string[-reverse.find(';'):-1]
    for posi in range(len(string)):
        if string[posi]==';':
            semic.append(posi)
    for positionx in range(len(semic)-1):
        word=string[semic[positionx]+1:semic[positionx+1]]
        separated.append(float(word))
    separated.append(float(lastword))
    return separated
def notuntillfloat(name):
    a = input(name)
    try:
        b= float(a)
    except ValueError:
        print('Please enter a float')
        a= float(notuntillfloat(name))
    return a
#notuntillfloat('Please enter the Target OD 600 ')
def notuntillint(name):
    a = input(name)
    try:
        b= int(a)
    except ValueError:
        print('Please enter an integer')
        a= int(notuntillint(name))
    return a
#notuntillint('Please enter the total number of samples you have ')
def notuntillindex(name,array):
    a = int(notuntillint(name))
    try:
        b= array[a-1]
    except IndexError:
        print('The index you entered is out of bound. Please enter the currect position')
        a= int(notuntillindex(name,array))
    return a
#notuntillindex('Please enter the number of samples you want to change ',[1,2,3,4,5])

In [4]:
#time calculator
starttime=input('Please enter the start time in 24h (hh:mm): ')
endtime=input('Please enter the end time (hh:mm): ')
st=starttime.split(':')
et=endtime.split(':')
hr=int(et[0])-int(st[0])
if hr<0:
    hr+=24
mins=int(et[1])-int(st[1])
if mins<0:
    mins+=60
    hr-=1
diff=mins/60
hr2=hr+diff
print('The time difference is: '+str(hr)+'hours '+str(mins)+'minutes')
print('The time difference for the following program is: '+str(hr2))

Please enter the start time in 24h (hh:mm): 17:45
Please enter the end time (hh:mm): 10:00
The time difference is: 16hours 15minutes
The time difference for the following program is: 16.25


In [10]:
## diluting culture
print('The Diluter V4.5.1---Created by Harry Wang')
print('If the program does not work, please restart it from the first cell.')
print('You can directly copy the result to Ecxel')
print('If you have enough confidence, please use The Diluter Advanced. Stop this program first.')
maxresonableod=6
targetod=float(notuntillfloat('Please enter the Target OD 600: '))
if targetod>maxresonableod:
    print('This OD is too large. Please check if you enter the wrong number')
    targetod=float(notuntillfloat('Please reenter the Target OD 600: '))
numofhours=float(notuntillfloat('Please enter the number of hours you will grow the culture (hours): '))
desiredvol=float(notuntillfloat('Please enter the added culture media volume (ml): '))
doublehr=float(notuntillfloat('Please enter the doubling time of your strain (hours): '))
totalsample=int(notuntillint('Please enter the total number of samples you have: '))
samplesize=float(notuntillfloat('Please enter the volume (mL) of samples you have: '))
datatable=[]
conditions=np.array([targetod,numofhours,desiredvol,doublehr,totalsample,samplesize])
conditions=np.reshape(conditions,[1,6])
array_to_html(conditions,row_names=['Conditions'],col_names=['Target OD 600','Number of hours for growth','Added culture volume','Doubling time','Number of samples','Sample volume'],title='Please Check The Conditions for Cultivation')
print('If you need to change the conditions, please restart the program')
for i in range(totalsample):
    d=i+1
    currentod=float(notuntillfloat('Please enter the Current OD 600 for sample %d: '%(d)))
    if currentod>maxresonableod:
        print('This OD is too large. Please check if you enter the wrong number')
        currentod=float(notuntillfloat('Please reenter the Current OD 600 for sample %d: '%(d)))
    amt=(desiredvol*targetod)/(currentod*2**(numofhours/doublehr)-targetod)*1000
    if (amt/1000)>samplesize:
        suggesthr=doublehr*math.log(desiredvol*(targetod/currentod)*1000/samplesize, 2)
        print('You do not have enough sample. Please restart the program and choose a larger number of hours ')
        print('The minimum hours you need is :',suggesthr)
    datatable.append([currentod,amt])
datatable=np.array(datatable)
print('The datatable table entered: ')
array_to_html(datatable, row_names=range(1,len(datatable)+1),col_names=['Current OD','Amount need to add (ul)'], title='The Data Entered')
#for item in datatable:
    #print(item)
change=input('Do you need to change any data? Please enter  \'y\' or \'n\': ')
while change!='n':
    num=int(notuntillint('Please enter the number of changes you want to make(include th change of conditions): '))
    for j in range(num):
        position = int(notuntillindex('Please enter the position of the sample you want to change(enter 0 if you want to change the conditions): ',datatable))
        if position == 0:
            print('You want to change the cultivation conditions')
            targetod=float(notuntillfloat('Please enter the Target OD 600: '))
            if targetod>maxresonableod:
                print('This OD is too large. Please check if you enter the wrong number')
                targetod=float(notuntillfloat('Please reenter the Target OD 600: '))
            numofhours=float(notuntillfloat('Please enter the number of hours you will grow the culture (hours): '))
            desiredvol=float(notuntillfloat('Please enter the added culture culture volume (ml): '))
            doublehr=float(notuntillfloat('Please enter the doubling time of your strain (hours): '))
            totalsample=int(notuntillint('Please enter the total number of samples you have: '))
            samplesize=float(notuntillfloat('Please enter the volume (mL) of samples you have: '))
            conditions=np.array([targetod,numofhours,desiredvol,doublehr,totalsample,samplesize])
            conditions=np.reshape(conditions,[1,6])
            array_to_html(conditions,row_names=['Conditions'],col_names=['Target OD 600','Number of hours for growth','Added culture volume','Doubling time','Number of samples','Sample volume'],title='Please Check The Conditions for Cultivation')
            for j in range(len(datatable)):
                datatable[j][1]=(desiredvol*(targetod/datatable[j][0]))/(2**(numofhours/doublehr))*1000
            array_to_html(datatable, row_names=range(1,len(datatable)+1),col_names=['Current OD','Amount need to add (ul)'], title='The new result under different conditions')
        else:
            print('The value you want to change is: '+'Sample '+ str(position)+': OD '+str(datatable[position-1][0]))
            newdata=float(notuntillfloat('Please enter the correct OD: '))
            if newdata>maxresonableod:
                print('This OD is too large. Please check if you enter the wrong number')
                newdata=float(notuntillfloat('Please reenter the correct OD: '))
            newvol=(desiredvol*(targetod/newdata))/(2**(numofhours/doublehr))*1000
            datatable[position-1,0]=newdata
            datatable[position-1,1]=newvol
            print('The changed result is: '+'Sample '+ str(position)+': New OD '+str(datatable[position-1][0]))
    array_to_html(conditions,row_names=['Conditions'],col_names=['Target OD 600','Number of hours for growth','Added culture volume','Doubling time','Number of samples','Sample volume'],title='Current Conditions for Cultivation')
    array_to_html(datatable, row_names=range(1,len(datatable)+1),col_names=['Current OD','Amount need to add (ul)'], title='The new result')
    change=input('Do you need to change any data? Please enter  \'y\' or \'n\': ')
array_to_html(conditions,row_names=['Conditions'],col_names=['Target OD 600','Number of hours for growth','Added culture volume','Doubling time','Number of samples','Sample volume'],title='The Final Conditions for Cultivation')
array_to_html(datatable, row_names=range(1,len(datatable)+1),col_names=['Current OD','Amount need to add (ul)'], title='The Final result')

#save data to file

now = datetime.now()
date_time = now.strftime("%m/%d/%Y_%H:%M:%S")
datetime1 = now.strftime("%m%d%Y_%H-%M")
path = os.getcwd()
try:
    os.makedirs(path+'\\'+'Dilution Data') # create destination directory, if needed (similar to mkdir -p)
except OSError:
    # The directory already existed, nothing to do
    pass
filen = path+'\\'+'Dilution Data'+'\\'+datetime1+' cell_dilution_result.txt'
file1 = open(filen,"w")
file1.write(date_time+'\n')
df1 = pd.DataFrame(conditions, index=['Conditions'], columns=['Target OD 600','Number of hours for growth','Added culture volume','Doubling time','Number of samples','Sample volume'])
df2 = pd.DataFrame(datatable, index=range(1,len(datatable)+1),columns=['Current OD','Amount need to add (ul)'])

file1.write("The Final Conditions for Cultivation \n")
file1.writelines(df1.to_string())
file1.write('\n')
file1.write("The Final Data for Cultivation \n")
file1.writelines(df2.to_string())

file1.close()

The Diluter V4.5.1---Created by Harry Wang
If the program does not work, please restart it from the first cell.
You can directly copy the result to Ecxel
If you have enough confidence, please use The Diluter Advanced. Stop this program first.
Please enter the Target OD 600: 1
Please enter the number of hours you will grow the culture (hours): 16.25
Please enter the added culture media volume (ml): 5
Please enter the doubling time of your strain (hours): 2.5
Please enter the total number of samples you have: 1
Please enter the volume (mL) of samples you have: 1


,Target OD 600,Number of hours for growth,Added culture volume,Doubling time,Number of samples,Sample volume
Conditions,1.0,16.25,5.0,2.5,1.0,1.0


If you need to change the conditions, please restart the program
Please enter the Current OD 600 for sample 1: 0.926
The datatable table entered: 


,Current OD,Amount need to add (ul)
1,0.926,60.377758


Do you need to change any data? Please enter  'y' or 'n': n


,Target OD 600,Number of hours for growth,Added culture volume,Doubling time,Number of samples,Sample volume
Conditions,1.0,16.25,5.0,2.5,1.0,1.0


,Current OD,Amount need to add (ul)
1,0.926,60.377758
